## Welcome!
Join me in exploring the top rated chess players in the world!  
Below is the code that leads to the creation of a **live** dataset.

In [45]:
# general imports:
import pandas as pd
import geopandas as gpd
import numpy as np
import requests

Now, we can proceed. To start, my idea is to build a dataframe of all of the top rated players, with info such as their _global rank_, their _name_, their _country_ represented, and their _ELO_ (playing strength.) <br>
<br>
### Part 1: Creating CSV & Player Selection
I am going to use selenium webdriver to convert an existing website into a **.csv** file, so that I can experiment with it here.


In [38]:
# plan: use webdriver to control web browser (chrome)
from selenium import webdriver

# using 'By', we can pick apart column names
from selenium.webdriver.common.by import By

# using chrome, loading page (load wait time)
import time

In [39]:
# step 1: set up webdriver
driver = webdriver.Chrome()

# step 2: open url
url = "https://www.365chess.com/top-chess-players.php"
driver.get(url)

# step 3: page loading (3 sec time to load the page)
time.sleep(3)

# step 4: look at rows
rows = driver.find_elements(By.CSS_SELECTOR, 'table tbody tr')

# step 5: create empty list;
# evaluate data, create columns, and properly assign
data = []
for row in rows:
    cols = row.find_elements(By.TAG_NAME, 'td')
    player = {
        "Rank": cols[0].text.strip(),
        "Name": cols[1].text.strip(),
        "ELO": cols[2].text.strip(),
        "Title": cols[3].text.strip(),
        # FIDE ID was listed here in slot[4], i purposely left it out.
        "Country": cols[5].text.strip(),
        "Games": cols[6].text.strip(),
        "Birth Year": cols[7].text.strip()
    }
    data.append(player)

# step 6: done! now save as csv
df = pd.DataFrame(data)
df.to_csv('topChessPlayers.csv', index=False)

driver.quit()

Great! We now have a .csv file of the world's top rated chess players. This dataset is live, so any time the code is ran, it will be updated by the second. Pretty neat. Let's take a look at the currrent **top 10** list:

In [40]:
# top 10 in the world:
df.head(10).style.hide(axis='index')

Rank,Name,ELO,Title,Country,Games,Birth Year
1,Magnus Carlsen,2832,GM,Norway,3527,1990
2,Hikaru Nakamura,2802,GM,United States,3354,1987
3,Fabiano Caruana,2793,GM,United States,3040,1992
4,Erigaisi Arjun,2778,GM,India,904,2003
5,Ian Nepomniachtchi,2767,GM,Russian Federation,2637,1990
6,Gukesh D,2766,GM,India,1017,2006
7,Nodirbek Abdusattorov,2762,GM,Uzbekistan,998,2004
8,Yi Wei,2762,GM,China,1112,1999
9,Viswanathan Anand,2751,GM,India,4011,1969
10,Alireza Firouzja,2751,GM,France,1158,2003


_Beautiful!_ Now, I want to create a prompt that requests the user to input certain info, and receive a quick summary of the player. Maybe at first start with something simple like the name, but afterwards allow the user to apply different filters. Let's try it.

In [51]:
import pandas as pd

df = pd.read_csv('topChessPlayers.csv')

# forcing user to enter either first or last name is too restrictive
# going to try to allow the user to do either.
name_input = input("Enter player's name (first, last, or both):").strip().lower()

# split name into separate words
name_parts = name_input.split()

# function to retrieve player's first and last names
def get_name(full_name):
    parts = full_name.strip().lower().split()
    return parts[0], parts[-1] # -1 is the last element in array

# important line: makes columns of first + last names, respectively
df['firstName'], df['lastName'] = zip(*df['Name'].apply(get_name))

# test statements, check to see if any players match
# creates a dataframe of all names that match (could be multiple)
if len(name_parts) == 1:
    matching_players = df[(df['firstName'] == name_parts[0]) | (df['lastName'] == name_parts[-1])]
elif len(name_parts) == 2:
    matching_players = df[(df['firstName'] == name_parts[0]) & (df['lastName'] == name_parts[-1])]
else:
    matching_players = pd.DataFrame()

# reset indicies to make selection process clear.
matching_players = matching_players.reset_index(drop=True)

# if one player is the match, good
# if multiple matches, print them out and make a choice
if not matching_players.empty:
    if len(matching_players) == 1:
        player = matching_players.iloc[0]
        age = 2024 - player['Birth Year']
        print(f"""The player you selected was {player['Name']}. Out of all the players in the world, {player['lastName'].capitalize()} currently holds position rank {player['Rank']}. {player['firstName'].capitalize()} was born in {player['Country']}, in the year {player['Birth Year']}. Since then, he has achieved the impressive ELO of {player['ELO']} through a whopping {player['Games']} hard-fought matches. At the age of {age}, He currently holds one of the highest achievable title in all of chess - a {player['Title']}.""")

    else:
        print("\nMultiple players share this name. Please make a selection below: (Enter 1, 2, 3, etc.)")
        for idx, player in matching_players.iterrows():
            print(f"{idx + 1}: {player['lastName'].capitalize()}, {player['firstName'].capitalize()} ({player['Country']})")
            
        selected_index = int(input("\nSelect a player: ")) - 1
        
        if selected_index >= 0 and selected_index < len(matching_players):
            selected_player = matching_players.iloc[selected_index]
            age = 2024 - selected_player['Birth Year']
            print(f"The player you selected was {selected_player['Name']}. Out of all the players in the world, {selected_player['lastName'].capitalize()} currently holds position rank {selected_player['Rank']}. {selected_player['firstName'].capitalize()} was born in {selected_player['Country']}, in the year {selected_player['Birth Year']}. Since then, he has achieved the incredible ELO of {selected_player['ELO']} through a stellar {selected_player['Games']} hard-fought matches. At the age of {age}, he currently holds one of the highest achievable title in all of chess - a {selected_player['Title']}.")
        else:
            print("Invalid selection.")

else: 
    print("No player found.")

print("\n")

Enter player's name (first, last, or both): nodirbek



Multiple players share this name. Please make a selection below: (Enter 1, 2, 3, etc.)
1: Abdusattorov, Nodirbek (Uzbekistan)
2: Yakubboev, Nodirbek (Uzbekistan)



Select a player:  1


The player you selected was Nodirbek Abdusattorov. Out of all the players in the world, Abdusattorov currently holds position rank 7. Nodirbek was born in Uzbekistan, in the year 2004. Since then, he has achieved the incredible ELO of 2762 through a stellar 998 hard-fought matches. At the age of 20, he currently holds one of the highest achievable title in all of chess - a GM.




Finally, after lots of coding and debugging, I was able to achieve what I wanted for the most part. There are lots of tweaks that could be added to make things flow better and for the descriptions to be more specific, but this will do for now. 
### Part 2: Data Retrieval (numbers)
Let's see what we can do with the data that we have, maybe some kind of charts, and if not, just some general info made available to the user upon request.